In [7]:
from ConfigParser import SafeConfigParser
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
from util import save_rdd, get_parser
import json

exp_dir = 'en-en'
target_language = 'en'
config = '/home/ellery/wikimedia/missing_articles/missing_articles.ini'

cp = SafeConfigParser()
cp.read(config)
base_dir = os.path.join(cp.get('general', 'local_data_dir'), exp_dir)
hadoop_base_dir = os.path.join(cp.get('general', 'hadoop_data_dir'), exp_dir)

In [8]:
def to_str(t):
    uid, contributions = t
    contributions = list(contributions)
    uname = contributions[0]['user']
    contributions = [{k: d[k] for k in ('id', 'page_title', 'num_edits', 'timestamp', 'bytes_added')} for d in contributions]
    contributions.sort(key = lambda x: x['timestamp'])
    obj = {'uid': uid, 'uname': uname, 'contributions': contributions}
    return json.dumps(obj)
        

In [9]:
names = ["language_code", "user_id", "user", "id","page_title","num_edits","timestamp", "bytes_added"]
contributions_file = os.path.join(cp.get('general', 'contributions_dir'), target_language)
contributions = sc.textFile(contributions_file).map(get_parser(names)).filter(lambda x: len(x) == 8)
contributions = contributions.map(lambda x: (x['user_id'], x)).groupByKey()
contributions = contributions.map(to_str)
save_rdd(contributions, base_dir , hadoop_base_dir, cp.get('eval', 'contributions'))

0
256
256
256
0
0
0
0
